In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget
#%matplotlib ipympl

#%reload_ext tensorboard
#%matplotlib qt

In [2]:
import pandas as pd
import os, pickle
import numpy as np
from pathlib import Path
import SimpleITK as sitk
import pydicom
from LN_malignancy_GNN.pytorch.run_model_torch import RunModel

/home/erichjs/miniconda3/envs/pytorch_gpu_v2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/erichjs/miniconda3/envs/pytorch_gpu_v2/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


In [3]:
data_path = Path('../../data/ln_malignancy')
fold_dir = data_path.joinpath('liyuan_folds')
dicom_dir = data_path.joinpath('All_Pt_Data')

In [4]:
fold_dict = {}
patient_list = []

In [5]:
for idx in range(5):
    fold_dict[f"fold_{idx}"] = np.unique([f.name.split('_')[0] for f in fold_dir.glob(f"F{idx+1}/label/*")])
    patient_list.extend(np.unique([f.name.split('_')[0] for f in fold_dir.glob(f"F{idx+1}/label/*")]))

In [6]:
preset_fold_list = []
for fold in fold_dict:
    preset_fold_list.append([patient_list.index(pat) for pat in fold_dict[fold]])

In [9]:
with open(data_path.joinpath('preset_folds_malignancy_liyuan_091024.pkl'), 'wb') as f:
    pickle.dump(preset_fold_list, f)
    f.close()

In [55]:
fold_file_ex = pd.read_pickle(data_path.joinpath('preset_folds_malignancy_081324.pkl'))

In [10]:
patients = pd.read_pickle(data_path.joinpath('features_and_labels_070924.pkl'))

In [11]:
list(patients.index.get_level_values(0).unique()) == patient_list

True

In [57]:
from LN_malignancy_GNN.pytorch.gen_params_torch_cfg import *
model_config['data_version'] = 'initial_rot5_30_balance_v1'
model_config['balance_classes'] = True

In [58]:
test_model = RunModel(model_config)

Seed set to 42


logs are located at: logs/20240813-161655
remember to set the data


In [59]:
test_model.set_data()

In [94]:
test_model.set_train_test_split()

In [30]:
test_model.data.processed_file_names

['graph_0_5723648.pt',
 'graph_1_5759052.pt',
 'graph_2_70017472.pt',
 'graph_3_70034190.pt',
 'graph_4_70248556.pt',
 'graph_5_70488002.pt',
 'graph_6_70502204.pt',
 'graph_7_70524131.pt',
 'graph_8_70553974.pt',
 'graph_9_70587154.pt',
 'graph_10_70595774.pt',
 'graph_11_70601955.pt',
 'graph_12_70624479.pt',
 'graph_13_70714954.pt',
 'graph_14_70763599.pt',
 'graph_15_70884300.pt',
 'graph_16_70896740.pt',
 'graph_17_70897649.pt',
 'graph_18_70952004.pt',
 'graph_19_71037733.pt',
 'graph_20_71101476.pt',
 'graph_21_71545860.pt',
 'graph_22_71605915.pt',
 'graph_23_71697467.pt',
 'graph_24_72037883.pt',
 'graph_25_72171929.pt',
 'graph_26_72190479.pt',
 'graph_27_72354212.pt',
 'graph_28_72432523.pt',
 'graph_29_72517194.pt',
 'graph_30_72535793.pt',
 'graph_31_72656583.pt',
 'graph_32_72679823.pt',
 'graph_33_72727445.pt',
 'graph_34_72744313.pt',
 'graph_35_72748349.pt',
 'graph_36_72759966.pt',
 'graph_37_72952628.pt',
 'graph_38_72958408.pt',
 'graph_39_73111324.pt',
 'graph_40_7

In [ ]:
n_pats = len(test_model.data.patients.index.get_level_values(0).unique())

test_idx = [ patients.index.get_level_values(0).unique().get_loc(pat.split('_')[0])+ n_pats *int(pat.split('_')[-1]) for pat in aug_fold_pats]

In [ ]:
for pat, idx in zip(aug_fold_pats, test_idx):
    print(pat, idx)

In [ ]:
len(test_model.data.patients.iloc[test_model.train_splits[0]].index.get_level_values(0).unique())

In [31]:
train_fold_pats = test_model.data.patients.iloc[test_model.train_splits[0]]

In [33]:
fold_pats = train_fold_pats.index.get_level_values(0).unique()

In [35]:
aug_idx = [file_name.split('_')[1] for file_name in test_model.data.processed_file_names if file_name.split('_')[2] in fold_pats and 'rotation' in file_name]

In [83]:
all_patients = list(test_model.data.patients.index.get_level_values(0).unique()) + list(test_model.data.aug_patients.index.get_level_values(0).unique()) + list(test_model.data.aug_pos_patients.index.get_level_values(0).unique())

In [85]:
test_str = 'graph_1429_92493024_pos_rotation_2.pt'

In [88]:
'_'.join(test_str.split('.')[0].split('_')[2:])

'92493024_pos_rotation_2'

In [93]:
fold_pats = test_model.data.patients.index.get_level_values(0).unique()[test_model.train_splits[0]]
print(fold_pats)

Index(['5723648', '5759052', '70017472', '70034190', '70248556', '70488002',
       '70502204', '70524131', '70553974', '70587154',
       ...
       '91321834', '91326601', '91335737', '91352953', '91362027', '91364773',
       '91383551', '91427333', '91437276', '91466920'],
      dtype='object', name='patients', length=114)


In [95]:
for idx in test_model.train_splits[0]:
    print(idx, all_patients[idx])

0 5723648
1 5759052
2 70017472
3 70034190
4 70248556
5 70488002
6 70502204
7 70524131
8 70553974
9 70587154
10 70595774
11 70601955
12 70624479
13 70714954
14 70763599
15 70884300
16 70896740
17 70897649
18 70952004
19 71037733
20 71101476
21 71545860
22 71605915
23 71697467
24 72037883
25 72171929
26 72190479
27 72354212
28 72432523
29 72517194
30 72535793
31 72656583
32 72679823
33 72727445
34 72744313
35 72748349
36 72759966
37 72952628
38 72958408
39 73111324
40 73221412
41 73236864
42 73340775
43 74323330
44 90016580
45 90033680
46 90063571
47 90329623
48 90345287
49 90355646
50 90377515
51 90393331
52 90401283
53 90404035
54 90425108
55 90446362
56 90479294
57 90505742
58 90533929
59 90538277
60 90542958
61 90551726
62 90563210
63 90569177
64 90604910
65 90605978
66 90607776
67 90615423
68 90623897
69 90647452
70 90651629
71 90655744
72 90663566
73 90716287
74 90725031
75 90755105
76 90779002
77 90782172
78 90811957
79 90815204
80 90832368
81 90863518
82 90873849
83 90888477
84 9

In [ ]:
for idx in:
    L.seed_everything(42)
    model.set_model()
    model.set_callbacks(idx)
    
    model.trainers[idx] = L.Trainer(
                    max_epochs=model.config['n_epochs'],
                    accelerator="auto",
                    devices=model.config['gpu_device'] if torch.cuda.is_available() else None,
                    logger=[L.loggers.CSVLogger(save_dir=os.path.join(model.log_dir, f"csvlog_fold_{idx}")), L.loggers.TensorBoardLogger(save_dir=os.path.join(model.log_dir, f"tb_fold_{idx}"))],
                    callbacks=model.callbacks,
                    #check_val_every_n_epoch = 1,
                    #auto_lr_find=True
    )
    model.trainers[idx].fit(model.model, datamodule=model.data_module_cross_val[idx])

In [ ]:
idx = 3
model.best_checkpoints[f"fold_{idx}"] = {callback.monitor: callback.best_model_path for callback in model.trainers[idx].callbacks if 'ModelCheckpoint' in callback.__class__.__name__}
for monitor, best_model in model.best_checkpoints[f"fold_{idx}"].items():
                model.val_metrics[monitor][idx] = model.trainers[idx].validate(model.trainers[idx].model, datamodule=model.data_module_cross_val[idx], ckpt_path=best_model)[0]
                model.test_metrics[monitor][idx] = model.trainers[idx].test(model.trainers[idx].model, datamodule=model.data_module_cross_val[idx], ckpt_path=best_model)[0]

In [107]:
test_arr = []
for idx in range(5):
    test_file = scipy.io.loadmat(model.data.data_path.joinpath(f'Radiomics_features/Radio F{idx+1}_ct_fea.mat').as_posix())
    tmp_arr = test_file['fin_fea_all']
    print(tmp_arr.shape)
    test_arr.append(tmp_arr)

(236, 62)
(319, 62)
(272, 62)
(260, 62)
(302, 62)


In [34]:
np.shape(test_arr)

(1389, 63)

In [29]:
for fold in model.folds:
    print(len(fold))

39
37
39
37
40


In [79]:
fold_1 = model.data.patients.loc[model.data.patients.index.get_level_values(0).unique()[model.folds[0]]]
print(fold_1)
print(test_arr[0][:, -6:])


                labels  pri_lat  ln_level  ln_lat  dist_to_pri  pri_location
patients nodes                                                              
5723648  1      [0, 1]      1.0       2.0     1.0    32.225766           1.0
         2      [1, 0]      1.0       2.0     1.0    55.780821           1.0
         3      [1, 0]      1.0       2.0     1.0    57.792733           1.0
         4      [0, 1]      1.0       2.0     1.0    32.225766           1.0
5759052  1      [1, 0]      1.0       2.0     1.0    61.532512           1.0
...                ...      ...       ...     ...          ...           ...
72958408 4      [1, 0]      1.0       2.0     1.0    63.287044           2.0
         5      [1, 0]      1.0       2.0     1.0    63.356136           2.0
         6      [1, 0]      1.0       2.0     1.0    74.035464           2.0
         7      [1, 0]      1.0       2.0     1.0    82.298238           2.0
         8      [0, 1]      1.0       2.0     1.0    60.301327           2.0

In [149]:
np.array([1,1]) + [3,1]

array([4, 2])

In [4]:
pat_arr = model.data.patients['labels'].values
sum_arr = np.array([0,0])
for pat in pat_arr:
    sum_arr += pat

sum_arr

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


NameError: name 'model' is not defined

In [80]:
order_dict = {value: key for key, value in enumerate(fold_1['dist_to_pri'])}

In [82]:
ordered_test_arr = np.array(sorted(test_arr[0], key=lambda x: order_dict[x[-3]]))

In [113]:
test_arr[1]

array([[ 0.26587244,  0.21879529,  0.24951311, ...,  0.        ,
        57.90509477,  3.        ],
       [ 0.26953774,  0.23553   ,  0.25926134, ...,  1.        ,
        46.97073557,  3.        ],
       [ 0.27221676,  0.238972  ,  0.25838952, ...,  0.        ,
        37.88469348,  3.        ],
       ...,
       [ 0.26664939,  0.22075208,  0.24552703, ...,  1.        ,
        74.08272403,  3.        ],
       [ 0.27334758,  0.23288532,  0.26237641, ...,  0.        ,
        46.82413907,  3.        ],
       [ 0.26344338,  0.21890603,  0.24075666, ...,  0.        ,
        80.84862398,  3.        ]])

In [73]:
len(np.unique(test_arr[1][:,-3]))

285

In [119]:
len(np.rec.find_duplicate(test_arr[1][:, -2]))

33

In [83]:
for idx, idx_str in enumerate(fold_1.index):
    try:
        print(fold_1.loc[idx_str, 'dist_to_pri'], ordered_test_arr[idx-1, -3])
    except:
        print('ran out of spaces')

32.225766088644036 60.30132668523969
55.780821076782296 55.780821076782296
57.79273310719956 57.79273310719956
32.225766088644036 32.225766088644036
61.53251173160413 32.225766088644036
40.50308630215727 61.53251173160413
59.247362810508285 40.50308630215727
75.2545679676656 59.247362810508285
59.247362810508285 59.247362810508285
75.2545679676656 75.2545679676656
37.16180835212409 75.2545679676656
51.74214916294065 37.16180835212409
50.28916384272063 51.74214916294065
55.21322305390259 50.28916384272063
39.51265620026069 55.21322305390259
60.87076473973364 39.51265620026069
56.236109396009965 60.87076473973364
46.446205442425544 46.446205442425544
47.35240226218729 47.35240226218729
69.43702182553626 69.43702182553626
45.35416188179427 45.35416188179427
55.362442142665635 55.362442142665635
56.236109396009965 56.236109396009965
62.465990746965666 56.236109396009965
48.93362851863736 62.465990746965666
46.195237849804386 48.93362851863736
53.660972782833525 46.195237849804386
39.852854

In [41]:
model.fold_dict = {}
for idx in range(5):
    model.fold_dict[f"fold_{idx}"] = [f.name.split('.')[0] for f in sorted(model.data.data_path.joinpath(model.config['fold_dir']).glob(f"F{idx+1}/label/*"))]

In [42]:
model.fold_dict

{'fold_0': ['5723648_1',
  '5723648_2',
  '5723648_3',
  '5723648_4',
  '5759052_1',
  '5759052_2',
  '5759052_3',
  '5759052_4',
  '5759052_5',
  '5759052_6',
  '70017472_1',
  '70034190_1',
  '70248556_1',
  '70248556_10',
  '70248556_11',
  '70248556_12',
  '70248556_13',
  '70248556_2',
  '70248556_3',
  '70248556_4',
  '70248556_5',
  '70248556_6',
  '70248556_7',
  '70248556_8',
  '70248556_9',
  '70488002_1',
  '70488002_2',
  '70488002_3',
  '70502204_1',
  '70502204_10',
  '70502204_2',
  '70502204_3',
  '70502204_4',
  '70502204_5',
  '70502204_6',
  '70502204_7',
  '70502204_8',
  '70502204_9',
  '70524131_1',
  '70524131_10',
  '70524131_2',
  '70524131_3',
  '70524131_4',
  '70524131_5',
  '70524131_6',
  '70524131_7',
  '70524131_8',
  '70524131_9',
  '70553974_1',
  '70553974_10',
  '70553974_11',
  '70553974_2',
  '70553974_3',
  '70553974_4',
  '70553974_5',
  '70553974_6',
  '70553974_7',
  '70553974_8',
  '70553974_9',
  '70587154_1',
  '70587154_2',
  '70587154_3',


In [44]:
node_idx = {}
for idx in range(5):
    node_idx[f"fold_{idx}"] = [model.data.patients.index.get_loc((pat.split('_')[0], int(pat.split('_')[1]))) for pat in model.fold_dict[f"fold_{idx}"]]

In [46]:
len(node_idx['fold_0'])

227

In [48]:
import pickle
fold_dict = model.fold_dict
patients = model.data.patients
with open(model.data.data_path.joinpath('node_index_no_large.pkl'), 'wb') as f:
    pickle.dump(node_idx, f)
    pickle.dump(fold_dict, f)
    pickle.dump(patients, f)
    f.close()
    